In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
import json
from transformers import EarlyStoppingCallback

with open('dataset.json', 'r', encoding='utf-8') as f:
  data = json.load(f)

texts = data['texts']
labels = data['lables']

labels = [int(label) for label in labels]


X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)


class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


train_dataset = TextDataset(X_train, y_train, tokenizer, max_len=128)
test_dataset = TextDataset(X_test, y_test, tokenizer, max_len=128)


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="tensorboard",
    save_total_limit=2,
    push_to_hub=False,
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)


trainer.train()


results = trainer.evaluate()
print("Результаты оценки:", results)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-9-fd69bf3530d4>:91: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.716200,0.635953,0.729167,0.628571
2,0.649500,0.565037,0.812500,0.830189
3,0.577400,0.449925,0.854167,0.862745
4,0.474700,0.544107,0.770833,0.813559
5,0.286000,0.376279,0.812500,0.816327
6,0.243200,0.342440,0.875000,0.875000


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.716200,0.635953,0.729167,0.628571
2,0.649500,0.565037,0.812500,0.830189
3,0.577400,0.449925,0.854167,0.862745
4,0.474700,0.544107,0.770833,0.813559
5,0.286000,0.376279,0.812500,0.816327
6,0.243200,0.342440,0.875000,0.875000
7,0.148800,0.334199,0.854167,0.857143
8,0.142300,0.326963,0.875000,0.880000
9,0.141600,0.331300,0.854167,0.857143
10,0.092500,0.334015,0.875000,0.875000


Результаты оценки: {'eval_loss': 0.3269629180431366, 'eval_accuracy': 0.875, 'eval_f1': 0.88, 'eval_runtime': 20.6415, 'eval_samples_per_second': 2.325, 'eval_steps_per_second': 0.145, 'epoch': 10.0}
